## 1 Million Brain Cells
**Author:** [Severin Dicks](https://github.com/Intron7)

To run this notebook please make sure you have a working environment with all nessaray dependencies. Run the [data_downloader](https://github.com/scverse/rapids_singlecell-notebooks/blob/abc4fc6f3fe7f85cbffb94e76d190cad0ae00a5f/data_downloader.ipynb) notebook first to create the AnnData object we are working with. In this example workflow we'll be looking at a dataset of 1000000 brain cells from  [Nvidia](https://github.com/clara-parabricks/rapids-single-cell-examples/blob/master/notebooks/1M_brain_cpu_analysis.ipynb).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scanpy as sc
import cupy as cp

import time
import rapids_singlecell as rsc
import anndata as ad
import warnings
import cugraph

warnings.filterwarnings("ignore")

/home/sdicks/micromamba/envs/rapids-25.10a/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import rmm
from rmm.allocators.cupy import rmm_cupy_allocator

rmm.reinitialize(
    managed_memory=False,  # Allows oversubscription
    pool_allocator=True,  # default is False
    devices=0,  # GPU device IDs to register. By default registers only GPU 0.
)
cp.cuda.set_allocator(rmm_cupy_allocator)

In [4]:
import gc

## Load and Prepare Data

We load the sparse count matrix from an `h5ad` file using Scanpy. The sparse count matrix will then be placed on the GPU. 

In [5]:
data_load_start = time.time()

In [6]:
%%time
adata = ad.read_zarr("zarr/1M.zarr/")

CPU times: user 7.2 s, sys: 2.07 s, total: 9.27 s
Wall time: 2.5 s


We now load the the AnnData object into VRAM.

In [7]:
%%time
rsc.get.anndata_to_GPU(adata)

CPU times: user 746 ms, sys: 6.8 s, total: 7.55 s
Wall time: 7.55 s


Verify the shape of the resulting sparse matrix:

In [8]:
adata.shape

(1000000, 27998)

In [9]:
data_load_time = time.time()
print("Total data load and format time: %s" % (data_load_time - data_load_start))

Total data load and format time: 10.12544870376587


## Preprocessing

In [10]:
preprocess_start = time.time()

### Quality Control

We perform a basic qulitiy control and plot the results

In [11]:
%%time
rsc.pp.flag_gene_family(adata, gene_family_name="MT", gene_family_prefix="mt-")

CPU times: user 2.94 ms, sys: 0 ns, total: 2.94 ms
Wall time: 2.87 ms


In [12]:
%%time
rsc.pp.calculate_qc_metrics(adata, qc_vars=["MT"])

CPU times: user 105 ms, sys: 18.1 ms, total: 123 ms
Wall time: 123 ms


### Filter

We filter the count matrix to remove cells with an extreme number of genes expressed.
We also filter out cells with a mitchondrial countent of more than 20%.

In [13]:
%%time
adata = adata[
    (adata.obs["n_genes_by_counts"] < 5000)
    & (adata.obs["n_genes_by_counts"] > 500)
    & (adata.obs["pct_counts_MT"] < 20)
].copy()

CPU times: user 155 ms, sys: 28.1 ms, total: 183 ms
Wall time: 183 ms


In [14]:
gc.collect()

523

Many python objects are not deallocated until garbage collection runs. When working with data that barely fits in memory (generally, >50%) you may need to manually trigger garbage collection to reclaim memory.

We also filter out genes that are expressed in less than 3 cells.

In [15]:
%%time
rsc.pp.filter_genes(adata, min_cells=3)

filtered out 5459 genes that are detected in less than 3 cells
CPU times: user 140 ms, sys: 3.92 ms, total: 144 ms
Wall time: 143 ms


We store the raw expression counts in the `.layer["counts"]`

In [16]:
adata.layers["counts"] = adata.X.copy()

In [17]:
adata.shape

(982490, 22539)

### Normalize

We normalize the count matrix so that the total counts in each cell sum to 1e4.

In [18]:
%%time
rsc.pp.normalize_total(adata, target_sum=1e4)

CPU times: user 5.88 ms, sys: 0 ns, total: 5.88 ms
Wall time: 5.87 ms


Next, we log transform the count matrix.

In [19]:
%%time
rsc.pp.log1p(adata)

CPU times: user 48.8 ms, sys: 1.93 ms, total: 50.7 ms
Wall time: 50.5 ms


### Select Most Variable Genes

Now we search for highly variable genes. This function only supports the flavors `cell_ranger` `seurat` `seurat_v3` and `pearson_residuals`. As you can in scanpy you can filter based on cutoffs or select the top n cells. You can also use a `batch_key` to reduce batcheffects.

In this example we use `seurat_v3` for selecting highly variable genes based on the raw counts in `.layer["counts"]`. 

In [20]:
%%time
rsc.pp.highly_variable_genes(
    adata, n_top_genes=5000, flavor="seurat_v3", layer="counts"
)

CPU times: user 219 ms, sys: 2.38 ms, total: 222 ms
Wall time: 221 ms


Now we safe this version of the AnnData as adata.raw.

Now we restrict our AnnData object to the highly variable genes.

In [21]:
%%time
adata = adata[:, adata.var["highly_variable"]].copy()
adata.X.sort_indices()

CPU times: user 710 ms, sys: 2.5 ms, total: 712 ms
Wall time: 712 ms


In [22]:
%%time
del adata.layers["counts"]
#rsc.get.anndata_to_CPU(adata, layer="counts")

CPU times: user 72 μs, sys: 37 μs, total: 109 μs
Wall time: 87 μs


In [23]:
adata.shape

(982490, 5002)

In [24]:
gc.collect()

568

Next we regress out effects of counts per cell and the mitochondrial content of the cells. As you can with scanpy you can use every numerical column in `.obs` for this.

In [ ]:
%%time
rsc.pp.regress_out(adata, keys=["total_counts", "pct_counts_MT"],batchsize="all")

CPU times: user 2 μs, sys: 1 μs, total: 3 μs
Wall time: 6.44 μs


### Scale

Finally, we scale the count matrix to obtain a z-score and apply a cutoff value of 10 standard deviations.

In [ ]:
%%time
rsc.pp.scale(adata, max_value=10, zero_center = True)

CPU times: user 16.3 ms, sys: 145 μs, total: 16.4 ms
Wall time: 16.2 ms


### Principal component analysis

We use PCA to reduce the dimensionality of the matrix to its top 100 principal components. We use the PCA implementation from cuml to run this. With `use_highly_variable = False` we save VRAM since we already subset the matrix to only HVGs.

In [27]:
%%time
rsc.pp.pca(adata, n_comps=100, use_highly_variable=False, zero_center=True)

Time taken to create gram matrix: 0.610736608505249 seconds
CPU times: user 896 ms, sys: 70.4 ms, total: 966 ms
Wall time: 966 ms


In [30]:
preprocess_time = time.time()
print("Total Preprocessing time: %s" % (preprocess_time - preprocess_start))

Total Preprocessing time: 3.063197612762451


## Clustering and Visualization

### Computing the neighborhood graph and UMAP

Next we compute the neighborhood graph using rsc.

Scanpy CPU implementation of nearest neighbor uses an approximation, while the GPU version calculates the exact graph. Both methods are valid, but you might see differences.

In [39]:
%%time
rsc.pp.neighbors(adata, n_neighbors=15, n_pcs=50, algorithm="nn_descent")

CPU times: user 32.8 s, sys: 1.73 s, total: 34.6 s
Wall time: 3.12 s


Next we calculate the UMAP embedding using rapdis.

In [32]:
%%time
rsc.tl.umap(adata, min_dist=0.3)

CPU times: user 830 ms, sys: 147 ms, total: 977 ms
Wall time: 977 ms


### Clustering

Next, we use the Louvain and Leiden algorithm for graph-based clustering.

In [33]:
%%time
rsc.tl.louvain(adata, resolution=0.6)

CPU times: user 1.38 s, sys: 167 ms, total: 1.55 s
Wall time: 1.55 s


In [34]:
%%time
rsc.tl.leiden(adata, resolution=1.0)

CPU times: user 1.19 s, sys: 55.6 ms, total: 1.25 s
Wall time: 1.25 s


## TSNE

In [35]:
%%time
rsc.tl.tsne(adata, n_pcs=40)

[2025-09-22 10:29:09.172] [CUML] [warning] # of Nearest Neighbors should be at least 3 * perplexity. Your results might be a bit strange...
CPU times: user 4.39 s, sys: 12.9 s, total: 17.3 s
Wall time: 17.3 s


## Diffusion Maps

In [ ]:
%%time
rsc.tl.diffmap(adata)

CPU times: user 2.32 s, sys: 65.7 ms, total: 2.38 s
Wall time: 2.38 s


After this you can use `X_diffmap` for `sc.pp.neighbors` and other functions. 

In [37]:
print("Total Processing time: %s" % (time.time() - preprocess_start))

Total Processing time: 28.09231424331665
